In [1]:
import numpy as np
import collections 
import random as rd
import math 
import time

## Introduccion

##### En este

## Implementacion

In [2]:
class puzzle:
    NORTH ,SOUTH,EAST ,WEST = 1,2,3,4
    board = None
    parent = None
    depth = None
    neighbours = None
    isMax = False
    score = 0
    
    def __init__(self,board,parent=None,isMax=True):
        self.board = board
        self.parent = parent
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0
        self.isMax = isMax
        self.getNeighbours()
    
    def getMoves(self,move):
        moves = []
        if move == self.NORTH:
            direction = (-1,0)
            #Recorremos las fichas de arriba hacia abajo
            for i in range(1,4):
                for j in range(4):
                    moves.append([(i,j)])
        
        if move == self.SOUTH:
            direction = (1,0)
            #Recorremos las fichas de abajo hacia arriba
            for i in reversed(range(0,3)):
                for j in range(4):
                    moves.append([(i,j)])
            
        if move == self.EAST:
            direction = (0,1)
            #Recorremos las fichas de izquierda a derecha
            for j in reversed(range(0,4)):
                for i in range(4):
                    moves.append([(i,j)])
                    
        if move == self.WEST:
            direction = (0,-1)
             #Recorremos las fichas de derecha a izquierda
            for j in range(0,4):
                for i in range(4):
                    moves.append([(i,j)])
        
        
        return moves,direction
    
    #Funcion que realiza los merges de los numeros iguales
    def mergeNumbers(self,obj,dest,board):
        
        if board[obj] == board[dest]:
            board[dest] =board[obj] + board[dest]
            board[obj] = 0
            return True
        return False
            
    def movePiece(self,src,dest,board):
        if dest[0] < 0 or dest[1] < 0  or dest[1] > 3 or dest[0] > 3:
            return False
        elif board[dest] != 0:
            return self.mergeNumbers(src,dest,board)
        
        else:
            board[dest] = board[src]
            board[src] = 0
            return True
    
    #Funcion que realiza los movimientos de las fichas
    def moveBoard(self,move):
        board = self.board.copy()
        moves,direction = self.getMoves(move)
        
        for mv in moves:
            src = mv[0]

            dest = src
            band = True
            while band:
                dest = (dest[0] + direction[0],dest[1] + direction[1])
                band = self.movePiece(src,dest,board)
                src = dest

        return board
        
    def getNeighbours(self):
        self.neighbours = []
        if self.isMax:
            for i in range(1,5):
                board = self.moveBoard(i)
                #El siguiente if nos permite podar un poco mas los vecinos.
                if not np.array_equal(self.board,board):
                    self.neighbours.append(board)
        else:
            self.getMinNeighbours()
    
    def getMinNeighbours(self):
        for i in range(4):
            for j in range(4):
                if self.board[i,j] == 0:
                    board1 =  self.board.copy()
                    board2 = self.board.copy()
                    board1[i,j] = 2
                    board2[i,j] = 4
                    self.neighbours.append(board1)
                    self.neighbours.append(board2)
                    
    def getPenalty(self):
        penalty = 0
        for i in range(4):
            for j in range(4):
                vecs = [[-1,0],[1,0],[0,-1],[0,1]]
                if self.board[i,j] !=0:
                    for vec in vecs:
                        desti = i+ vec[0]
                        destj = j+ vec[1]
                        if desti >= 0 and desti <= 3 and destj >= 0 and destj <= 3 and self.board[desti,destj] != 0:
                            penalty += abs(self.board[i,j] - self.board[desti,destj] )
        return penalty
    
    def getScore(self):
        w = [
        [[0,1,2,4],
        [-1,0,1,2],
        [-2,-1,0,1],
        [-4,-2,-1,0]]
        ,
        [[4,2,1,0],
        [2,1,0,-1],
        [1,0,-1,-2],
        [0,-1,-2,-4]]
        ,
        [[0,-1,-2,-4],
        [1,0,-1,-2],
        [2,1,0,-1],
        [4,2,1,0]]
        ,
        [[-4,-2,-1,0],
        [-2,-1,0,1],
        [-1,0,1,2],
        [0,1,2,4]]
        ]
        values = -np.inf
        for wi in w:
            values = max(values,np.sum(self.board * wi)+np.count_nonzero(self.board == 0))


        return values
    
    def getScore2(self):
        if len(self.neighbours) == 0:
            return -np.inf

        w = [[[2*3,2*4,2*11,2*12],
             [2*2,2*5,2*10,2*13],
             [2*1,2*6,2*9,2*14],
             [2*0,2*7,2*8,2*15]]
             
            ,[[2*3,2*2,2*1,2*0],
             [2*4,2*5,2*6,2*7],
             [2*11,2*10,2*9,2*8],
             [2*12,2*13,2*14,2*15]]
             
            ,[[2*15,2*14,2*13,2*12],
             [2*8,2*9,2*10,2*11],
             [2*7,2*6,2*5,2*4],
             [2*0,2*1,2*2,2*3]]
            
            ,[[2*15,2*8,2*7,2*0],
             [2*14,2*9,2*6,2*1],
             [2*13,2*10,2*5,2*2],
             [2*12,2*11,2*4,2*3]]
             
             ,[[2*0,2*7,2*8,2*15],
             [2*1,2*6,2*9,2*14],
             [2*2,2*5,2*10,2*13],
             [2*3,2*4,2*11,2*12]]
             
             ,[[2*12,2*13,2*14,2*15],
             [2*11,2*10,2*9,2*8],
             [2*4,2*5,2*6,2*7],
             [2*3,2*2,2*1,2*0]]
             
             ,[[2*12,2*11,2*4,2*3],
             [2*13,2*10,2*5,2*2],
             [2*14,2*9,2*6,2*1],
             [2*15,2*8,2*7,2*0]]
             
             ,[[2*0,2*1,2*2,2*3],
             [2*7,2*6,2*5,2*4],
             [2*8,2*9,2*10,2*11],
             [2*15,2*14,2*13,2*12]]
            ]
            
            
        values = -np.inf
        for i in w:
            values = max(np.sum(self.board * i),values)
        zero = (np.count_nonzero(self.board == 0) * 10)
        return values


    def isGoal(self):
        return np.any(self.board == 2048)
    
    

In [3]:
def CreaEstado():
    board = np.zeros((4,4))
    possvalues = [2,4]
    for i in range(2):
        randpos = (rd.randint(0,3),rd.randint(0,3))
        randvalue = rd.randint(0,1)
        if board[randpos] == 0:
            board[randpos] = possvalues[randvalue]
        else:
            i-=1
    
    return puzzle(board)

startObj = CreaEstado()
print startObj.board

[[0. 2. 0. 0.]
 [0. 0. 4. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
def alpha_beta(node,actualdepth,maxdepth,isMax,alpha,beta):

    finalnode = None
    if actualdepth == maxdepth:
        return node.getScore()
    if len(node.neighbours) == 0:
        return -np.inf

    if isMax:
        bestVal = MIN
        for neighbour in node.neighbours:
            newNode = puzzle(neighbour,node,False)
            value = alpha_beta(newNode,actualdepth+1,maxdepth,False,alpha,beta)
            bestVal = max(value,bestVal)
            alpha = max(alpha,bestVal)
            
            if alpha >= beta:
                break
    
        return bestVal
    
    else:
        bestVal = MAX
        for neighbour in node.neighbours:
            newNode = puzzle(neighbour,node,True)
            value = alpha_beta(newNode,actualdepth+1,maxdepth,True,alpha,beta)
            bestVal = min(value,bestVal)
            beta = min(beta,bestVal)
                
            if  alpha >= beta:
                break
        
        return bestVal
        
        
MAX, MIN = 10000,-10000
val = alpha_beta(startObj,0,3,True,MIN,MAX)

print val

31.0


In [32]:
def alpha_beta(node,maxdepth,isMax,alpha,beta):

    finalnode = None
    if node.depth == maxdepth or node.isGoal() or len(node.neighbours) == 0:
        return node,node.getScore()
    
    if isMax:
        bestVal = -10000
        for neighbour in node.neighbours:
            
            newNode = puzzle(neighbour,node,False)
            ansnode,value = alpha_beta(newNode,maxdepth,False,alpha,beta)
            bestVal = max(value,bestVal)
            alpha = max(alpha,bestVal)
            
            if bestVal == value:
                finalnode = ansnode
            
            if beta <= alpha:
                break
        return finalnode,bestVal
    else:
        bestVal = 10000
        for neighbour in node.neighbours:
            newNode = puzzle(neighbour,node,True)
            ansnode,value = alpha_beta(newNode,maxdepth,True,alpha,beta)
            
            bestVal = min(value,bestVal)                
            beta = min(beta,bestVal)
            
            if bestVal == value:
                finalnode = ansnode
                
            if beta <= alpha:
                break
        
        return finalnode,bestVal
        
        
        
node,val = alpha_beta(startObj,2,True,-10000,10000)

print node.board,val

[[0. 4. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 4.]] 1416.0


## Resultados

In [7]:
start = CreaEstado()
node = start
it = 0

while not node.isGoal() and len(node.neighbours) > 0:
    possnode = None
    util = -np.inf
    print node.board,"\n"
    for i,neig in enumerate(node.neighbours):
        newnode = puzzle(neig,node)
        if np.count_nonzero(node.board==0) <= 4: 
            val = alpha_beta(newnode,0,4,True,MIN,MAX)
        else:
            val = alpha_beta(newnode,0,2,True,MIN,MAX)
        if val >= util:
            possnode = i
            util =val

    it+=1
    
    #generamos al siguiente nodo
    minnode = puzzle(node.neighbours[possnode],node,False)
    node = puzzle(minnode.neighbours[rd.randint(0,len(minnode.neighbours)-1)],node)

print node.board
print "Cantidad de movimientos: ",it




[[2. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [2. 0. 0. 2.]] 

[[0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 2. 0. 2.]] 

[[4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 2. 0.]
 [4. 2. 0. 0.]] 

[[0. 0. 0. 4.]
 [4. 0. 0. 0.]
 [0. 0. 0. 2.]
 [0. 0. 4. 2.]] 

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [4. 0. 0. 0.]
 [4. 0. 4. 8.]] 

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 4.  0.  0.  2.]
 [16.  0.  0.  0.]] 

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 4.  2.  0.  0.]
 [16.  2.  0.  0.]] 

[[ 4.  4.  0.  0.]
 [16.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  2.]] 

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [16.  4.  4.  2.]] 

[[ 0.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [ 0.  0.  0.  4.]
 [ 0. 16.  8.  2.]] 

[[ 2.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [16.  8.  2.  0.]] 

[[ 2.  8.  2.  0.]
 [ 8.  0.  0.  0.]
 [16.  0.  0.  0.]
 [ 0.  2.  0.  0.]] 

[[ 0.  4.  0.  0.]
 [ 2.  0.  0.  0.]
 [ 8.  8.  0.  0.]
 [16.  2.  2

[[  0.   0.   0.   0.]
 [  4.   0.   0.   2.]
 [  0.   0.   8.   2.]
 [  0. 256.  16.   8.]] 

[[  0.   4.   0.   0.]
 [  4.   2.   0.   0.]
 [  8.   2.   0.   0.]
 [256.  16.   8.   0.]] 

[[  4.   0.   0.   0.]
 [  4.   2.   0.   0.]
 [  8.   2.   0.   0.]
 [256.  16.   8.   2.]] 

[[ 16.   4.   8.   2.]
 [256.  16.   0.   0.]
 [  0.   0.   4.   0.]
 [  0.   0.   0.   0.]] 

[[ 16.   4.   8.   2.]
 [256.  16.   4.   0.]
 [  0.   0.   0.   0.]
 [  2.   0.   0.   0.]] 

[[  0.   0.   4.   0.]
 [ 16.   0.   0.   0.]
 [256.   4.   8.   0.]
 [  2.  16.   4.   2.]] 

[[ 16.   4.   4.   2.]
 [256.  16.   8.   4.]
 [  2.   0.   4.   0.]
 [  0.   0.   0.   0.]] 

[[  0.  16.   8.   2.]
 [256.  16.   8.   4.]
 [  0.   0.   2.   4.]
 [  0.   0.   4.   0.]] 

[[256.  32.  16.   2.]
 [  0.   2.   2.   8.]
 [  0.   0.   4.   0.]
 [  0.   0.   0.   0.]] 

[[256.  32.  16.   2.]
 [  4.   8.   0.   0.]
 [  4.   0.   0.   0.]
 [  4.   0.   0.   0.]] 

[[256.  32.  16.   2.]
 [  8.   8.   0.   0.]
 [  